In [29]:
import numpy as np
import pandas as pd
from py_vollib_vectorized import get_all_greeks
import plotly as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

In [30]:
def plot():
    fig = make_subplots(5,1)
    x = np.arange(500,2000,100)

    trace0 = go.Scatter(x = x,
                    y = pnl,
                    mode='lines',
                    name='pnl')
    trace1 = go.Scatter(x = x,
                    y = delta,
                    mode='lines',
                    name='delta')
    trace2 = go.Scatter(x = x,
                    y = gamma,
                    mode='lines',
                    name='gamma')
    trace3 = go.Scatter(x = x,
                    y = theta,
                    mode='lines',
                    name='theta')
    trace4 = go.Scatter(x = x,
                    y = gamma,
                    mode='lines',
                    name='vega')
    fig.add_trace(trace0,1,1)
    fig.add_trace(trace1,2,1)
    fig.add_trace(trace2,3,1)
    fig.add_trace(trace3,4,1)
    fig.add_trace(trace4,5,1)

    py.offline.iplot(fig)

In [31]:
# 牛市看涨
# call：平值多头 + 虚值空头

s = 1000
k1,k2 = 1000,1500
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (i - k1 ) if i > k1 else 0
    pnl2 = 0 if i<k2 else (k2-i)
    pl = pnl1 + pnl2
    pnl.append(pl)

# delta
delta=[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('c',i,k1,t,0,sigma)['delta'][0]-get_all_greeks('c',i,k2 ,t,0,sigma)['delta'][0])

# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    gamma.append(get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0]-get_all_greeks('c',i,k2 ,t,0,sigma)['gamma'][0])
    theta.append(get_all_greeks('c',i,k1,t,0,sigma)['theta'][0]-get_all_greeks('c',i,k2 ,t,0,sigma)['theta'][0])
    vega.append(get_all_greeks('c',i,k1,t,0,sigma)['vega'][0]-get_all_greeks('c',i,k2 ,t,0,sigma)['vega'][0])

plot()

In [32]:
# 熊市看跌
# put：平值多头 + 虚值空头

s = 1000
k1,k2 = 1000,500
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (k1 -i) if i < k1 else 0
    pnl2 = 0 if i > k2 else (i-k2)
    pl = pnl1 + pnl2
    pnl.append(pl)

# delta
delta=[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('p',i,k1,t,0,sigma)['delta'][0]-get_all_greeks('p',i,k2 ,t,0,sigma)['delta'][0])

# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    gamma.append(get_all_greeks('p',i,k1,t,0,sigma)['gamma'][0]-get_all_greeks('p',i,k2 ,t,0,sigma)['gamma'][0])
    theta.append(get_all_greeks('p',i,k1,t,0,sigma)['theta'][0]-get_all_greeks('p',i,k2 ,t,0,sigma)['theta'][0])
    vega.append(get_all_greeks('p',i,k1,t,0,sigma)['vega'][0]-get_all_greeks('p',i,k2 ,t,0,sigma)['vega'][0])

plot()

In [34]:
# 跨式多头
s = 1000
k1,k2 = 1000,1000
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (i - k1) if i > k1 else 0
    pnl2 = (k2-i) if i < k2 else 0
    pl = pnl1 + pnl2
    pnl.append(pl)

# Gamma, Theta, Vega
delta, gamma, theta, vega = [],[],[],[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['delta'][0])
    gamma.append(get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['gamma'][0])
    theta.append(get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['theta'][0])
    vega.append(get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['vega'][0])

plot()

In [35]:
# 跨式空头
s = 1000
k1,k2 = 1000,1000
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = -(i - k1) if i > k1 else 0
    pnl2 = -(k2 - i) if i < k2 else 0
    pl = pnl1 + pnl2
    pnl.append(pl)

# Gamma, Theta, Vega
delta, gamma, theta, vega = [],[],[],[]
for i in range(500,2000,100):
    delta.append(-get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['delta'][0])
    gamma.append(-get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['gamma'][0])
    theta.append(-get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['theta'][0])
    vega.append(-get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['vega'][0])

plot()

In [36]:
# 蝶式多头
# long ITM call + short OTM call X2 + long OTM call

s = 1000
k1,k2,k3 = 800,1000,1200
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (i - k1) if i > k1 else 0
    pnl2 = 0 if i < k2 else (k2-i) * 2
    pnl3 = (i - k3) if i > k3 else 0
    pl = pnl1 + pnl2 + pnl3
    pnl.append(pl)

# delta
delta=[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['delta'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['delta'][0])

# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    gamma.append(get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['gamma'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0])
    theta.append(get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['theta'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['theta'][0])
    vega.append(get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['vega'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['vega'][0])

plot()

In [37]:
# 蝶式空头
# short ITM call + long ATM call X2 + short OTM call

s = 1000
k1,k2,k3 = 800,1000,1200
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (k1 - i) if i > k1 else 0
    pnl2 = (i - k2) * 2 if i > k2 else 0
    pnl3 = (k3 - i) if i > k3 else 0
    pl = pnl1 + pnl2 + pnl3
    pnl.append(pl)

# delta
delta=[]
for i in range(500,2000,100):
    delta.append(-get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['delta'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['delta'][0])

# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    gamma.append(-get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['gamma'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0])
    theta.append(-get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['theta'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['theta'][0])
    vega.append(-get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['vega'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['vega'][0])

plot()